In [173]:
import csv, string, nltk, collections, os
import functools, operator
import pandas as pd
import numpy as np

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import pairwise_distances

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.sparse import csr_matrix
from scipy.stats import wasserstein_distance
from scipy import stats

from timeit import default_timer as timer

from pyemd import emd

import matplotlib.pyplot as plt

from operator import itemgetter

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/andra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/andra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/andra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Phase one
Computing distribution clusters

### Read data

In [2]:
# Read data
data_imdb = pd.read_csv('movies3/csv_files/imdb.csv')
data_rt = pd.read_csv('movies3/csv_files/rotten_tomatoes.csv')

# Clean data
data_imdb = data_imdb.fillna(0)
data_rt = data_rt.fillna(0)
data_rt = data_rt.replace({'Rating': ['N', '.']}, {'Rating': 0})

# Store data for future processing 
data1 = data_imdb
data2 = data_rt

# Get the columns
columns1 = data1.columns
columns2 = data2.columns

#### Dummy numerical data based on the imdb data

In [3]:
dataX = pd.DataFrame(data1.Rating)
dataX.columns = ['Score']
dataX = dataX.reindex(np.random.permutation(dataX.index)).reset_index()
dataX['Rating'] = data1.Rating

#### Small test data and CountVectorizer test

In [4]:
d1 = "Obama speaks to the media in Illinois"
d2 = "The President addresses the press in Chicago"

vect = CountVectorizer(stop_words="english").fit([d1, d2])
print("Features:",  ", ".join(vect.get_feature_names()))

Features: addresses, chicago, illinois, media, obama, president, press, speaks


#### TPC-H Data

In [3]:
# customer = pd.read_csv('tpch/customer.csv', sep='|', header=None)
# lineitem = pd.read_csv('tpch/lineitem.csv', sep='|', header=None)
# nation = pd.read_csv('tpch/nation.csv', sep='|', header=None)
# orders = pd.read_csv('tpch/orders.csv', sep='|', header=None)
# part = pd.read_csv('tpch/part.csv', sep='|', header=None)
# region = pd.read_csv('tpch/region.csv', sep='|', header=None)
# supplier = pd.read_csv('tpch/supplier.csv', sep='|', header=None)
# partsupp = pd.read_csv('tpch/partsupp.csv', sep='|', header=None)

customer = pd.read_csv('tpch/customer.csv', sep='|')
# lineitem = pd.read_csv('tpch/lineitem.csv', sep='|')
# nation = pd.read_csv('tpch/nation.csv', sep='|')
# orders = pd.read_csv('tpch/orders.csv', sep='|')
part = pd.read_csv('tpch/part.csv', sep='|')
# region = pd.read_csv('tpch/region.csv', sep='|')
# supplier = pd.read_csv('tpch/supplier.csv', sep='|')
# partsupp = pd.read_csv('tpch/partsupp.csv', sep='|')

### Get the google news vocabulary and store it as a memory map - Used for Word_EMD
The function returns an array:
- result[0] the data map
- result[1] the vocabulary map

Download the data from: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [7]:
def get_vocabulary():
    if not os.path.exists("data/embed.dat"):
        print("Caching word embeddings in memmapped format...")
        from gensim.models import KeyedVectors
        wv = KeyedVectors.load_word2vec_format(
            "GoogleNews-vectors-negative300.bin.gz", binary=True)
        wv.init_sims()
        from tempfile import mkdtemp
        import os.path as path
        dat_file = path.join(mkdtemp(), 'embed.dat')
        vocab_file = path.join(mkdtemp(), 'embed.vocab')
        fp = np.memmap(dat_file, dtype=np.double, mode='w+', shape=wv.vectors_norm.shape)
        fp[:] = wv.vectors_norm[:]
        with open(vocab_file, "w+") as f:
            for _, w in sorted((voc.index, word) for word, voc in wv.vocab.items()):
                print(w, file=f)
        del fp, wv

    W = np.memmap(dat_file, dtype=np.double, mode="r", shape=(3000000, 300))
    with open(vocab_file) as f:
        vocab_list = map(str.strip, f.readlines())

    return [W, {w: k for k, w in enumerate(vocab_list)}]

In [6]:
result = get_vocabulary()
W = result[0]
vocab_dict = result[1]

Caching word embeddings in memmapped format...


### Algorithm 1 from the paper 
Algorithmically identifying the cutoff EMD threshold for a column C, given a global threshold. 

In [175]:
def compute_cutoff_threshold(C, threshold):
    t = {}
    t['e'] = threshold
    t['c'] = 0
    C.append(t)
    C = sorted(C, key = lambda i: i['e']) 
    cutoff = 0
    gap = 0.0
    i = 0
    while C[i + 1]['e'] <= threshold:
        if gap < (C[i+1]['e'] - C[i]['e']):
            gap = C[i+1]['e'] - C[i]['e']
            cutoff = C[i]['e']
        i += 1

    return cutoff      

### Get the neighbors of a column
The neighborhood NC of column C consists of all columns C′ with EMD(C, C′) ≤ cutoff.

In [176]:
def get_neighbors(C, cutoff):
    return [i['c'] for i in C if i['e'] <= cutoff]

### Algorithm 2 from the paper
Compute distribution graph and distribution cluster. 

In [177]:
def compute_distribution_clusters(data, columns, threshold):
    graph = {}
    A = {}
#     vocab_dict = get_vocabulary()
  
    for i in range(0, len(columns)):
        for j in range(i + 1, len(columns)):
#             try:
#                 e = wasserstein_distance(data[columns[i]], data[columns[j]])
#             except ValueError:
#                 e = word_emd(data[columns[i]], data[columns[j]], vocab_dict)
            e = quantile_emd(data[columns[i]], data[columns[j]])
            item_j = {}
            item_j['e'] = e
            item_j['c'] = columns[j]
            if columns[i] not in A:
                A[columns[i]] = []
            A[columns[i]].append(item_j)

            item_i = {}
            item_i['e'] = e
            item_i['c'] = columns[i]
            if columns[j] not in A:
                A[columns[j]] = []
            A[columns[j]].append(item_i)
        graph[columns[i]] = []
    
    for i in range(len(columns)):
        theta = compute_cutoff_threshold(A[columns[i]], threshold)
        Nc = get_neighbors(A[columns[i]], theta)

        for c in Nc:
            graph[columns[i]] = c
      
    return graph


#### Prepare data

In [178]:
customer = pd.read_csv('tpch/customer.csv', sep='|', nrows=20)
lineitem = pd.read_csv('tpch/lineitem.csv', sep='|', nrows=20)
nation = pd.read_csv('tpch/nation.csv', sep='|', nrows=20)
orders = pd.read_csv('tpch/orders.csv', sep='|', nrows=20)
part = pd.read_csv('tpch/part.csv', sep='|', nrows=20)
region = pd.read_csv('tpch/region.csv', sep='|', nrows=20)
supplier = pd.read_csv('tpch/supplier.csv', sep='|', nrows=20)
partsupp = pd.read_csv('tpch/partsupp.csv', sep='|', nrows=20)

data = pd.concat([customer, lineitem, nation, orders, part, region, supplier, partsupp], axis=1)

In [358]:
threshold = 0.14
C = compute_distribution_clusters(data, data.columns, threshold)

{8324.07, 6.34, 3857.34, 5266.3, 794.47, 6819.74, 2866.83, 7603.4, 2753.54, 3396.49, 711.56, 4681.03, 7498.12, 8914.71, 7638.57, 9561.95, 2788.52, -272.6, 5494.43, 121.65}
['000000015', 8324.07, '000000016', 6.34, 'customer', '000000018', '000000019', 3857.34, 5266.3, '000000013', 794.47, '000000010', 6819.74, '000000004', '000000002', '000000005', 2866.83, 7603.4, '000000011', '000000012', '000000007', '000000009', 2753.54, 3396.49, 711.56, '000000020', '000000017', 4681.03, 7498.12, 8914.71, 7638.57, 9561.95, 2788.52, '000000001', '000000003', '000000006', -272.6, '000000014', 5494.43, '000000008', 121.65]
[1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0
 17.0 18.0 19.0 20.0 '-272.6' '121.65' '2753.54' '2788.52' '2866.83'
 '3396.49' '3857.34' '4681.03' '5266.3' '5494.43' '6.34' '6819.74'
 '711.56' '7498.12' '7603.4' '7638.57' '794.47' '8324.07' '8914.71'
 '9561.95' 'customer']
{'000000015', '000000016', 'customer', '000000018', '000000019', '000000013', '000000

UnboundLocalError: local variable 'ranks2' referenced before assignment

### Word EMD - NOT working
Word EMD algortihm according to: https://vene.ro/blog/word-movers-distance-in-python.html which is based on the paper: http://mkusner.github.io/publications/WMD.pdf

In [11]:
def word_emd(d1, d2, vocab_dict):
    corpus = d1 + d2
    l1 = len(d1)

    vect = CountVectorizer(stop_words="english").fit(corpus)
    W_ = W[[vocab_dict[w] if w in vocab_dict else vocab_dict['unk'] for w in vect.get_feature_names()] ]
    D_ = euclidean_distances(W_)
    D_ = D_.astype(np.double)
    D_ /= D_.max() 
    
    v_ = vect.transform(corpus)
    v_1 = v_[:l1,:]
    v_2 = v_[l1:, :]
    
    print(v_1.shape)
    print(v_1)
    print(v_2.shape)
    print(v_2)
    
    v_1 = v_1.toarray().ravel()
    v_2 = v_2.toarray().ravel()
    v_1 = v_1.astype(np.double)
    v_2 = v_2.astype(np.double)
    v_1 /= v_1.sum()
    v_2 /= v_2.sum()
    
    print(v_1.shape)
    print(v_2.shape)
    print(D_.shape)
    
    from pyemd import emd

    return [emd(v_1, v_2, D_), vect.get_feature_names()]

#### Test the methods given different data

In [21]:
threshold = 0.14

g = compute_distribution_clusters(dataX, dataX.columns, threshold)
# g = compute_distribution_clusters(data1, data1.columns, threshold)

print(g)

{'index': [], 'Score': 'Rating', 'Rating': 'Score'}


In [114]:
# d1 = 'Massimo Morini'
# d2 = 'Beppe Mecconi'

# e = word_emd(data1['Director'].astype(str).tolist(), data1['Creators'].astype(str).tolist(), vocab_dict)
e = word_emd([d1], [d2], vocab_dict)
print(e)
# data1['Creators'].tolist() + data1['Director'].tolist()

(1, 8)
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 7)	1
(1, 8)
  (0, 0)	1
  (0, 1)	1
  (0, 5)	1
  (0, 6)	1
(8,)
(8,)
(8, 8)
[0.744844, ['addresses', 'chicago', 'illinois', 'media', 'obama', 'president', 'press', 'speaks']]


In [13]:
d11 = nltk.word_tokenize(d1.lower())
d12 = nltk.word_tokenize(d2.lower())

print(d11)
print(d12)

uni = set(d11).union(set(d12))
sort = sorted(list(uni))
print(sort)
del sort[3]
del sort[8]
del sort[8]

wmap = {key: i for (i, key) in enumerate(sort)}
print(wmap)

['obama', 'speaks', 'to', 'the', 'media', 'in', 'illinois']
['the', 'president', 'addresses', 'the', 'press', 'in', 'chicago']
['addresses', 'chicago', 'illinois', 'in', 'media', 'obama', 'president', 'press', 'speaks', 'the', 'to']
{'addresses': 0, 'chicago': 1, 'illinois': 2, 'media': 3, 'obama': 4, 'president': 5, 'press': 6, 'speaks': 7}


### Copied wasserstein distance code from python repo

In [16]:
def _cdf_distance(p, u_values, v_values, u_weights=None, v_weights=None):
    r"""
    Compute, between two one-dimensional distributions :math:`u` and
    :math:`v`, whose respective CDFs are :math:`U` and :math:`V`, the
    statistical distance that is defined as:
    .. math::
        l_p(u, v) = \left( \int_{-\infty}^{+\infty} |U-V|^p \right)^{1/p}
    p is a positive parameter; p = 1 gives the Wasserstein distance, p = 2
    gives the energy distance.
    Parameters
    ----------
    u_values, v_values : array_like
        Values observed in the (empirical) distribution.
    u_weights, v_weights : array_like, optional
        Weight for each value. If unspecified, each value is assigned the same
        weight.
        `u_weights` (resp. `v_weights`) must have the same length as
        `u_values` (resp. `v_values`). If the weight sum differs from 1, it
        must still be positive and finite so that the weights can be normalized
        to sum to 1.
    Returns
    -------
    distance : float
        The computed distance between the distributions.
    Notes
    -----
    The input distributions can be empirical, therefore coming from samples
    whose values are effectively inputs of the function, or they can be seen as
    generalized functions, in which case they are weighted sums of Dirac delta
    functions located at the specified values.
    References
    ----------
    .. [1] Bellemare, Danihelka, Dabney, Mohamed, Lakshminarayanan, Hoyer,
           Munos "The Cramer Distance as a Solution to Biased Wasserstein
           Gradients" (2017). :arXiv:`1705.10743`.
    """
#     u_values, u_weights = _validate_distribution(u_values, u_weights)
#     v_values, v_weights = _validate_distribution(v_values, v_weights)

    u_sorter = np.argsort(u_values)
    v_sorter = np.argsort(v_values)

    all_values = np.concatenate((u_values, v_values))
    all_values.sort(kind='mergesort')

    # Compute the differences between pairs of successive values of u and v.
    deltas = np.diff(all_values)

    # Get the respective positions of the values of u and v among the values of
    # both distributions.
    u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right')
    v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')

    # Calculate the CDFs of u and v using their weights, if specified.
    if u_weights is None:
        u_cdf = u_cdf_indices / u_values.size
    else:
        u_sorted_cumweights = np.concatenate(([0],
                                              np.cumsum(u_weights[u_sorter])))
        u_cdf = u_sorted_cumweights[u_cdf_indices] / u_sorted_cumweights[-1]

    if v_weights is None:
        v_cdf = v_cdf_indices / v_values.size
    else:
        v_sorted_cumweights = np.concatenate(([0],
                                              np.cumsum(v_weights[v_sorter])))
        v_cdf = v_sorted_cumweights[v_cdf_indices] / v_sorted_cumweights[-1]

    # Compute the value of the integral based on the CDFs.
    # If p = 1 or p = 2, we avoid using np.power, which introduces an overhead
    # of about 15%.
    if p == 1:
        return np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))
    if p == 2:
        return np.sqrt(np.sum(np.multiply(np.square(u_cdf - v_cdf), deltas)))
    return np.power(np.sum(np.multiply(np.power(np.abs(u_cdf - v_cdf), p),
                                       deltas)), 1/p)

#### Same algorithm as above
Tried to make it work for strings - NOT WORKING

In [191]:
u_sorter = np.argsort(u)
print(u_sorter)
v_sorter = np.argsort(v)
print(v_sorter)

all_values = np.concatenate((u, v))
all_values = np.array(list(set(all_values)))

all_args = np.argsort(all_values)
print(all_args)
print(all_values)
all_values.sort(kind='mergesort')
print(all_values)

# uni = set(d11).union(set(d12))
# sort = sorted(list(uni))
# print(list(uni))
# print(sort)

# arg = np.argsort(list(uni))
# print(arg)

fv = np.vectorize(nltk.edit_distance)
deltas = fv(all_values[:-1], all_values[1:])
# deltas = np.diff(all_args)
# deltas = np.diff(all_values)
print(deltas)

u_cdf_indices = np.array(u)[u_sorter].searchsorted(all_values[:-1], 'right')
v_cdf_indices = np.array(v)[v_sorter].searchsorted(all_values[:-1], 'right')

# print(all_values[:-1])

# print(np.array(u)[u_sorter])
# print(v[v_sorter])

# print(u_cdf_indices)
# print(v_cdf_indices)

u_cdf = u_cdf_indices / np.array(u).size
v_cdf = v_cdf_indices / np.array(v).size

# print(u_cdf)
# print(v_cdf)

np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))
# np.sum(np.abs(u_cdf - v_cdf))

[0 1 2]
[1 0 2]
[3 0 4 2 1]
['i' 'popcorn' 'like' 'bike' 'iou']
['bike' 'i' 'iou' 'like' 'popcorn']
[3 2 3 7]


2.0

### Quantile histogram algorithm - NOT working
Copied from https://cobr.io/blog/implementing-a-multi-column-foreign-key-discovery-algorithm.html


In [14]:
import math
def quantilehistogram(values, numbins=256):
        try:
            lists = [list(t) for t in zip(*values)] # unpack pairs of values into a list of lists
        except:
            lists = [list(t) for t in zip(values)]

        if len(lists) == 0: # empty column...
            return None

        hists = []
        for l in lists:
            binsize = int(math.sqrt(len(l)))
            if binsize >= 500:
                binsize = 499

            hist = []
            bins = []
            try:
                # print('trying as is..')
                sum(l)
                hist, bins = np.histogram(l, bins=binsize, density=True) # sqrt to improve accuracy for larger tables
            except:
                try:
                    # print('trying to cast to ints..')
                    castlist = [ int(value) for value in l ]
                    hist, bins = np.histogram(castlist, bins=binsize, density=True)
                except:
                    # print('trying as is hashed..')
                    hashedlist = [ hash(value) for value in l ]
                    hist, bins = np.histogram(hashedlist, bins=binsize, density=True)
                # c = collections.Counter(l)
                # rhist = list(map((lambda x: x/len(l)), list(c.values()))) # for each quantile (map) divide by total number of records to get probability
                # rbins = list(c)

                # for i in range(numbins):
                # 	if i < len(rhist) and i < len(rbins):
                # 		hist.append(rbins[i])
                # 		bins.append(rhist[i])
                # 	else:
                # 		hist.append(0)
                # 		bins.append(0)
                # bins.append(0)
            hists.append((list(hist), list(bins)))

        return hists

### Quantile EMD - NOT working
Source: https://cobr.io/blog/implementing-a-multi-column-foreign-key-discovery-algorithm.html

In [15]:
def q_emd(qfk, qpk):
    emdscore = 0
    for i in range(len(qfk)):
        fkhist = qfk[i][0]
        pkhist = qpk[i][0]

        fkbins = qfk[i][1]
        pkbins = qpk[i][1]
#         print(np.transpose(np.array(fkhist)))
        emdscore += emd(np.transpose(np.array(fkhist)), np.transpose(np.array(pkhist)), 
                        np.ascontiguousarray(np.array([fkbins[0:-1], pkbins[0:-1]]).T))
    emdscore = emdscore/len(qfk[0])
    return emdscore

In [54]:
# hist1 = quantilehistogram(data1['Title'][:100])
# hist2 = quantilehistogram(data2['Title'][:100])

hist1 = quantilehistogram(d11)
hist2 = quantilehistogram(d12)

# hist2 = quantilehistogram(dataX['Score'][:100])
# q_emd(hist1, hist2)

In [ ]:
q_emd(hist1, hist2)
# np.array([hist1[1][1][0:-1], hist2[1][1][0:-1]]).T.shape
# np.concatenate((np.array(hist1[1][1][0:-1]), np.array(hist2[1][1][0:-1])))

# dataX['Score'][:100]

### Quantile EMD 
The final version of emd according to the paper

In [357]:
def quantile_emd(column1, column2):
    quantile = 1
#     punct = '!"&\'()*+,-./:;<=>?[\\]^_`{|}~'
#     sign = '#$@%'
    table = str.maketrans({key: ' ' for key in string.punctuation})
#     table = str.maketrans({key: ' ' for key in punct})
#     table.update(str.maketrans({key: '' for key in sign}))
    
    if type(column1) is not np.ndarray:
        column1 = np.array(column1)
    if type(column2) is not np.ndarray:
        column2 = np.array(column2) 

    # if data is string, process it (apply lowercase, remove punctuation and tokenize the strings)
    if (type(column1[0]) is str) or (type(column1[0]) is np.str_):
        column1 = column1.astype(str)
        column1 = np.chararray.translate(column1, table)
        column1 = np.char.lower(column1)
        column1 = [nltk.word_tokenize(token) for token in column1]
        column1 = np.concatenate(column1).ravel()
        
    if (type(column2[0]) is str) or (type(column2[0]) is np.str_):   
        column2 = column2.astype(str)
        column2 = np.chararray.translate(column2, table)
        column2 = np.char.lower(column2)
        column2 = [nltk.word_tokenize(token) for token in column2]
        column2 = np.concatenate(column2).ravel()
        
    # get the unique values
    set1 = set(column1)
    set2 = set(column2)
    
    # compute the union of the 2 columns
    set_union = list(set1.union(set2))
    
    # sort the values in lexicographi/numeric order
    try:
        condition = np.char.isnumeric(set_union)
        numeric_values = np.sort(list(set(np.extract(condition, set_union).astype(float)))).astype(np.object)
        string_values = np.sort(np.extract(~condition, set_union)).astype(np.object)
        sorted_set = np.append(numeric_values, string_values, axis=0)
    except:
        sorted_set = sorted(set_union)

    # rank the words in lexicographic/numeric order
    wmap = {key: i for (i, key) in enumerate(sorted_set)}
    ranks = np.array(list(wmap.values()))
    ranks_l = len(ranks)

    # get the ranks for all the data for each column
    ranks1 = sorted([(itemgetter(*[float(text) if str(text).isdigit() else text for text in list(set1)])(wmap))])
    l1 = len(ranks1)

    try:
        ranks2 = sorted([(itemgetter(*[float(text) if str(text).isdigit() else text for text in list(set2)])(wmap))])
    except:
        print(set2)
        print(set_union)
        print(sorted_set)
        print(set1)
    # get the bin edges by using 1-quantile
    bin_edges1 = stats.mstats.mquantiles(ranks1, np.array(range(0, l1 + 1, quantile)) / l1)

    # compute the histogram for both columns
    hist1, bins1 = np.histogram(ranks1, bins=bin_edges1)
    hist2, bins2 = np.histogram(ranks2, bins=bin_edges1)
    
    # find the distance matrix between each word
    # computation of D for 1/4 of data ~ 1 min
    D = pdist(ranks.reshape(-1, 1), 'minkowski', p=1.)
    
    # computation of emd for 1/4 of data ~ 15 mins
    e = emd(hist1 / ranks_l, hist2 / ranks_l, squareform(D)) / ranks_l

    return e

In [348]:
a = [123.78]

np.char.isnumeric(a)

TypeError: isnumeric is only available for Unicode strings and arrays

#### Dataset used in the paper
Using only 1/4 for testing purposes and because my laptop runs out of memory

In [ ]:
# psc = np.array(part['P_Comment'][0:10]).astype(str)
# psc = np.array(partsupp[4][0:10])
psc = part['P_PartKey']
psc = np.array(psc[0:int(len(psc) / 4)])
# psc = np.array(psc)[0:10]
# print(type(psc[1]))

# custc = customer['C_Comment']
# custc = customer[7][0:10]
custc = customer['C_CustKey']
custc = np.array(custc[0:int(len(custc) / 4)])
# custc = np.array(custc[0:10])
# print(type(custc[1]))

#### Test the emd 

In [121]:
quantile_emd(psc, custc)

KeyboardInterrupt: 